In [8]:
import open3d as o3d
# from open3d.web_visualizer import draw
import numpy as np
import os
import json

In [10]:
dirname = """/home/rizo/mipt_ccm/vlsat_inference/point_clouds_test"""

In [15]:

pcds = {}
for framename in os.listdir(dirname):
    with open(os.path.join(dirname, framename)) as f:
        frame = json.load(f)
    number_of_large_objects = 0 
    print(f"Processing frame: {framename}, {frame.keys()}")
    for i, obj in enumerate(frame['objects']):
        print(f"obj keys: {obj.keys()}")
        if obj["id"] not in pcds:
            pcds[obj["id"]] = []
        pc_array = np.array(obj["point_cloud"])
        pose =  [
                    np.round(np.mean(pc_array[:, 0]),2),
                    np.round(np.mean(pc_array[:, 1]),2),
                    np.round(np.mean(pc_array[:, 2]),2),
                ]
        pcds[obj["id"]].append(pose)

Processing frame: test_shapes.json, dict_keys(['objects'])
obj keys: dict_keys(['id', 'name', 'point_cloud'])
obj keys: dict_keys(['id', 'name', 'point_cloud'])


In [16]:
for track_id in pcds:
    for pose in pcds[track_id]:
        
        print(track_id, pose)

0 [-0.18, -0.16, 0.33]
1 [0.0, 0.0, 1.19]


In [17]:
from collections import Counter
def pcd_denoise_dbscan(pcd: o3d.geometry.PointCloud, eps=0.02, min_points=10) -> o3d.geometry.PointCloud:
    ### Remove noise via clustering
    pcd_clusters = pcd.cluster_dbscan(
        eps=eps,
        min_points=min_points,
    )
    
    # Convert to numpy arrays
    obj_points = np.asarray(pcd.points)
    #obj_colors = np.asarray(pcd.colors)
    pcd_clusters = np.array(pcd_clusters)

    # Count all labels in the cluster
    counter = Counter(pcd_clusters)

    # Remove the noise label
    if counter and (-1 in counter):
        del counter[-1]

    if counter:
        # Find the label of the largest cluster
        most_common_label, _ = counter.most_common(1)[0]
        
        # Create mask for points in the largest cluster
        largest_mask = pcd_clusters == most_common_label

        # Apply mask
        largest_cluster_points = obj_points[largest_mask]
        #largest_cluster_colors = obj_colors[largest_mask]
        
        # If the largest cluster is too small, return the original point cloud
        if len(largest_cluster_points) < 5:
            return pcd

        # Create a new PointCloud object
        largest_cluster_pcd = o3d.geometry.PointCloud()
        largest_cluster_pcd.points = o3d.utility.Vector3dVector(largest_cluster_points)
        
        pcd = largest_cluster_pcd
        
    return pcd

In [18]:
dirname = "point_clouds-2/point_clouds/"
framename = "frame_1.json"
with open(os.path.join(dirname, framename)) as f:
    frame = json.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'point_clouds-2/point_clouds/frame_1.json'

In [43]:
tracked_objects = []
for obj in frame["tracked_objects"]:
    if obj['class_id'] == 1 and len(obj['point_cloud']) > 1000:
        tracked_objects.append(obj)
    print(len(obj['point_cloud']), obj["class_id"], obj["tracking_id"], obj["pose"])
    

8561 1 0 [1.217146589845437, 0.4739201112582472, -0.16513834905526797]
22426 1 1 [1.0195534155715849, -0.2916883106121823, -0.17452518878210538]
40 0 2 [2.0295629718295216, -0.2005763133122287, -0.7444377363721665]
18488 1 3 [1.1381738629158495, 0.05979025773894107, -0.15925896826165598]
8700 1 4 [1.0389699183548167, -0.24019952444992446, -0.014888865596165323]
820 1 5 [1.8881230092774821, -0.5665642856546798, -0.6846687536191339]


In [49]:
for obj in frame["tracked_objects"]:
    pcd_o3d = o3d.geometry.PointCloud()
    pcd_o3d.points = o3d.utility.Vector3dVector(obj['point_cloud'])
    pcd = pcd_denoise_dbscan(pcd_o3d )
    obj['point_cloud'] = np.array(pcd.points)


In [67]:
tracked_objects = []
for obj in frame["tracked_objects"]:
    if obj['class_id'] == 1 and len(obj['point_cloud']) > 1000:
        pose = obj["pose"]
        #size = [
        #    np.max(obj['point_cloud'][:,2]) - np.min(obj['point_cloud'][:,2]),
        #    np.max(obj['point_cloud'][:,0]) - np.min(obj['point_cloud'][:,0]),
        #    np.max(obj['point_cloud'][:,1]) - np.min(obj['point_cloud'][:,1]),
        #]
        size = [0.3, 0.3, 0.15]
        nx, ny, nz = (16, 16, 16)
        x = np.linspace(pose[0] - size[0]/2, pose[0] + size[0]/2, nx)
        y = np.linspace(pose[1] - size[1]/2, pose[1] + size[1]/2, ny)
        z = np.linspace(pose[2] - size[2]/2, pose[2] + size[2]/2, nz)
        xv, yv, zv = np.meshgrid(x, y, z)
        print(x.shape, y.shape, z.shape)
        #print(xv)
        print(xv.shape, yv.shape, zv.shape)
        #print(xv.flatten())
        grid_pc = np.stack((xv.flatten(), yv.flatten(), zv.flatten()), axis=1)
        print(grid_pc.shape)
        #print(grid_pc)

        tracked_objects.append(
            {
                'point_cloud': grid_pc
            }
        )

(16,) (16,) (16,)
(16, 16, 16) (16, 16, 16) (16, 16, 16)
(4096, 3)
(16,) (16,) (16,)
(16, 16, 16) (16, 16, 16) (16, 16, 16)
(4096, 3)
(16,) (16,) (16,)
(16, 16, 16) (16, 16, 16) (16, 16, 16)
(4096, 3)
(16,) (16,) (16,)
(16, 16, 16) (16, 16, 16) (16, 16, 16)
(4096, 3)


In [68]:
colors = [
    (0.85, 0.33, 0.10),
    (0.47, 0.67, 0.19),
    (0.29, 0.56, 0.89),
    (0.64, 0.08, 0.18),
    (0.10, 0.75, 0.61),
    (0.98, 0.84, 0.21),
    (0.30, 0.30, 0.30),
    (0.68, 0.47, 0.83),
    (0.55, 0.34, 0.29),
    (0.12, 0.72, 0.77),
    (0.92, 0.28, 0.58),
    (0.14, 0.42, 0.75),
    (0.40, 0.76, 0.65),
    (0.90, 0.49, 0.13),
    (0.22, 0.64, 0.82)
]

colors_array = []

for i, obj in enumerate(tracked_objects):
    color_array = np.ones_like(obj['point_cloud'])
    color_array[:,0] = colors[i][0]
    color_array[:,1] = colors[i][1]
    color_array[:,2] = colors[i][2]
    colors_array.append(color_array)


[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 192.168.0.x:45604 (eno1)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45667 (ztly5r4qjp)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 10.147.18.x:42940 (ztqu3aetcl)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 10.43.71.x:56541 (ztvli23lch)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45667 (ztly5r4qjp)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 10.147.18.x:42940 (ztqu3aetcl)
[8634:732][2365614] (stun_port.cc:96): Binding request timed out from 10.43.71.x:56541 (ztvli23lch)


In [69]:
pcd_data = [
    obj['point_cloud']
    for obj in tracked_objects
]
pcd = np.concatenate(pcd_data, axis=0)
colors = np.concatenate(colors_array, axis=0)

In [19]:
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(pcd)
pcd_o3d.colors = o3d.utility.Vector3dVector(colors)
draw(pcd_o3d)

NameError: name 'pcd' is not defined

In [13]:
data_path = "./point_clouds"
pcd_path = "point_cloud_20240806_001539_id_7.npy"
data_dict_1= np.load(os.path.join(data_path, pcd_path), allow_pickle=True).item()["point_cloud"]
pcd_path = "point_cloud_20240806_001540_id_7.npy"
data_dict_2 = np.load(os.path.join(data_path, pcd_path), allow_pickle=True).item()["point_cloud"]

(15329, 3)


In [53]:
distance_x = np.mean(data_dict_1[:,0]) - np.mean(data_dict_2[:,0])
distance_y = np.mean(data_dict_1[:,1]) - np.mean(data_dict_2[:,1])
distance_z = np.mean(data_dict_1[:,2]) - np.mean(data_dict_2[:,2])

data_dict_1[:, 0] = data_dict_1[:, 0] - distance_x
data_dict_1[:, 1] = data_dict_1[:, 1] - distance_y
data_dict_1[:, 1] = data_dict_1[:, 1] - distance_z

In [54]:
np.mean(data_dict_1[:,0])

-0.35952908270454353

In [55]:
colors_1 = np.zeros_like(data_dict_1)
colors_1[:,1] = 1 + colors_1[:,1]
colors_2 = np.zeros_like(data_dict_2)
colors_2[:,2] = 1 + colors_2[:,2] 

In [56]:
pcd = np.concatenate((data_dict_1, data_dict_2), axis=0)
colors = np.concatenate((colors_1, colors_2), axis=0)

In [57]:
pcd_o3d = o3d.geometry.PointCloud()
pcd_o3d.points = o3d.utility.Vector3dVector(pcd)
pcd_o3d.colors = o3d.utility.Vector3dVector(colors)

In [58]:
draw(pcd_o3d)

[Open3D INFO] Window window_7 created.


WebVisualizer(window_uid='window_7')

[Open3D INFO] Sending init frames to window_7.


[653:727][4150639] (stun_port.cc:96): Binding request timed out from 192.168.0.x:44238 (eno1)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45835 (ztly5r4qjp)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 10.147.18.x:54575 (ztqu3aetcl)
[653:727][4150639] (stun_port.cc:96): Binding request timed out from 10.43.71.x:40285 (ztvli23lch)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 10.147.18.x:54575 (ztqu3aetcl)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 172.24.249.x:45835 (ztly5r4qjp)
[653:730][4150639] (stun_port.cc:96): Binding request timed out from 10.43.71.x:40285 (ztvli23lch)
